In [1]:
%matplotlib qt
from tqdm import tqdm_notebook #########!!!!!!!!!!!!!!!!!
from __future__ import print_function, division

import numpy as np

from mpl_toolkits.mplot3d import Axes3D  # needed for 3d-plots
import matplotlib.pyplot as plt

from scipy.optimize import brute, fminbound, minimize_scalar
# brute force minimizing
from itertools import combinations
from ipywidgets import interact, interactive, VBox, fixed, interact_manual, IntSlider
#from numba import njit
from sklearn.neighbors import KDTree
from matplotlib.ticker import MultipleLocator, FormatStrFormatter




def distances(coord):
    '''
    calculate all appearing distances in an array of coordinates
    '''
    n = coord.shape[0]
    result = []
    for i in range(n):
        for j in range(i):
            result.append(np.linalg.norm((coord[i] - coord[j])))
    return np.array(result)


def dist_pnt(coord, point):
    '''
    calculate all distances of an array of coordinates to a single point
    '''
    return np.sqrt((coord[:, 0]-point[0])**2 +
                   (coord[:, 1]-point[1])**2 +
                   (coord[:, 2]-point[2])**2)


def first_val(hist, bin_edges, bound):
    '''
    get the first distance value that appears more often then given in "bounds"

    as input you can use the output of numpy.histogram
    '''
    return bin_edges[:-1][hist > bound][0]


def ang_to_norm(phi, psi):
    '''
    get a normal-vector for the hessian-normal form of a plane given by two
    angles (phi - angle to positive z-axes, psi - angle to positive x-axes)
    '''
    return np.array([np.sin(phi)*np.cos(psi),
                     np.sin(phi)*np.sin(psi),
                     np.cos(phi)])


def dist_to_plane(points, phi, psi, dzero):
    '''
    calculate the distance of a point to a plane, where the plane is given by
    phi, psi, and dzero (distance to the origin)
    '''
    points = np.array(points, ndmin=2)
    norm = ang_to_norm(phi, psi)
    result = np.empty(points.shape[0])

#    for i in range(points.shape[0]):
#        result[i] = abs(np.dot(norm, points[i]) - dzero) !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    result = np.abs(norm[0]*points[:, 0] +
                    norm[1]*points[:, 1] +
                    norm[2]*points[:, 2] - dzero)
    return result


def dist_to_plane_sgn(points, phi, psi, dzero):
    '''
    calculate the distance of a point to a plane, where the plane is given by
    phi, psi, and dzero (distance to the origin) (with sign)
    '''
    points = np.array(points, ndmin=2)
    norm = ang_to_norm(phi, psi)
    result = np.empty(points.shape[0])

#    for i in range(points.shape[0]):
#        result[i] = abs(np.dot(norm, points[i]) - dzero)

    result = (norm[0]*points[:, 0] +
              norm[1]*points[:, 1] +
              norm[2]*points[:, 2] - dzero)
    return result


def dist_to_line(points, norm, vec):
    '''
    distance of points to a given line by x = t*n + v where n is a normal-vec

    returns : distances and the parameter t of each nearest point

    optimal t is given by: t_min = norm*(point - vec)
    '''
    para = (norm[0]*(points[:, 0] - vec[0]) +
            norm[1]*(points[:, 1] - vec[1]) +
            norm[2]*(points[:, 2] - vec[2]))

    result = np.sqrt((points[:, 0] - norm[0]*para - vec[0])**2 +
                     (points[:, 1] - norm[1]*para - vec[1])**2 +
                     (points[:, 2] - norm[2]*para - vec[2])**2)

    return result, para


def rescale_basis_vec(points, norm, vec, eps=None):
    '''
    return the points on the line within an eps-surrounding
    '''
    if eps is None:
        dist_coord = distances(points)
        # use the histogram to find the minimum distance between the points
        min_dist = first_val(*np.histogram(dist_coord, bins=points.shape[0]))
        # set the epsilon for checking if point in plane
        eps = min_dist/10

    dists, paras = dist_to_line(points, norm, vec)
    # select the t-parameters of the nearest points on line
    paras = paras[dists < eps]
    paras = np.sort(paras)
    # the consecutive distances of the params
    cons_dist = paras[1:]-paras[:-1]

#    print("consecutive distances", cons_dist)

    return np.mean(cons_dist)


def no_of_points(in_arg, points, eps, blist=None, angl=np.pi/3.0):
    '''
    get the number of points whose distance to the given plane is less than eps

    you can "blacklist" certain angles for psi and phi in blist
    with "angl" you can specify the angle around the blacklisted angles
    (stnd value: 45°)

    if an angle-combination is blacklisted, 0 is returned
    '''
    if blist is None:
        blist = []

    phi, psi, dzero = in_arg

    # iterate over blacklisted angles
    for bphi, bpsi in blist:
        if (bphi-angl < phi < bphi+angl) and (bpsi-angl < psi < bpsi+angl):
            return 0

    dist = dist_to_plane(points, phi, psi, dzero)
    # result is negative so the minimizer is working as a maximizer
    return -len(dist[dist < eps])


def center_unit(points):
    '''
    center and scale a list of points to unitcube and give the shift and
    scaling

    shifting is realized by the average position

    return : newpoints, scale, shift
    '''
    coord = points.copy()
    shift = np.mean(coord, axis=0)
    for i in range(3):
        coord[:, i] -= shift[i]
    scale = np.max(np.abs(coord), axis=0)
    for i in range(3):
        coord[:, i] /= scale[i]

    return coord, scale, shift


def affin_hessian(norm, dist, matrix, vec=np.array([0., 0., 0.])):
    '''
    transformation of a plane given in hessian normal form (n*x=d) under a
    affine transformation: y = T(x) = M*x + v
    '''
    # inverse matrix of the transformation
    inv_mat = np.linalg.inv(matrix)
    # the new normal-vector n_new = n*M^-1
    new_norm = np.squeeze(np.dot(np.array(norm, ndmin=2), inv_mat))
    abs_norm = np.linalg.norm(new_norm)
    new_norm = new_norm/abs_norm

    new_dist = dist/abs_norm + np.dot(new_norm, vec)

    return new_norm, new_dist


def estimate_planes(points, iterations=4,
                    error_angle=np.pi/3.0,
                    grid_angle=np.pi/90):
    '''
    performe estimations to lay planes in a cloud of points

    with iterations you can specifiy the amount of planes

    with error_angle you can set the angle of minimal difference between planes

    with grid_angle you can specify the grid-size for the brute-force estimate
    '''
    coord = points.copy()
    # empty list for results
    resbrute = []
    # scale the coords
    coord, scale, shift = center_unit(coord)
    #print("scale", scale)
    #print("shift", shift)
    # get the distances within the coords
    dist_coord = distances(coord)
    
    
    # use the histogram to find the minimum distance between the points
    min_dist = first_val(*np.histogram(dist_coord, bins=coord.shape[0]), bound = bounds)
    # set a maximal range where the plane can move from the origin
    max_dist = np.max(dist_coord)/8.0
    # set the epsilon for checking if point in plane
    eps = min_dist/8.0
    #print("min_dist", min_dist)
    #print("max_dist", max_dist)
    #print("eps", eps)
    # initialize the black-list
    blist = []
    for i in range(iterations):
        print("----- iteration: {}".format(i))
        res = brute(no_of_points,
                    ranges=(slice(0., np.pi/2., grid_angle),  # 90° in 2° steps
                            slice(0., 2.*np.pi, grid_angle),  # 360° in 2° step
                            slice(-max_dist, max_dist, eps/2.5)),
                    args=(coord, eps, blist, error_angle),
                    disp=False)
        blist.append(res[:2])
        #print("phi, psi, dist", res)
        resbrute.append(res)

    # components of the affine backtransformation for the coords
#    trans_mat = np.diag(1/scale)
    trans_mat = np.diag(scale)
    trans_vec = shift

    # get the normal-vectors and dists of the planes from their angles
    norms = []
    dists = []
    for res in resbrute:
        norms.append(ang_to_norm(res[0], res[1]))
        dists.append(res[2])

    out = []

    # backtransform the planes to the original coords
    for norm, dist in zip(norms, dists):
        out.append(affin_hessian(norm, dist, trans_mat, trans_vec))

    return out


def plot_planes(coord, planes, vectors=None, origin=None, cubesize = 6):
    '''
    plot the choord-cloud and the planes given as a list of norms and dists
    '''
    # create the figure for plotting
    fig = plt.figure()
    
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')
    # create the first axis
    ax = fig.add_subplot(111, projection='3d')
    # plot the points
    ax.scatter(coord[:, 0], coord[:, 1], coord[:, 2], c='k' )
    
    
    ax.set_xlabel("kz [1/nm]")
    ax.set_ylabel("kx [1/nm]")
    ax.set_zlabel("ky [1/nm]")

    # get the axes-limits to cut the plane afterwards
    x_lim = ax.get_xlim()
    y_lim = ax.get_ylim()
    z_lim = ax.get_zlim()

    # set 30 steps for the plane in x and y direction
    x_stp = (x_lim[1] - x_lim[0])/30
    y_stp = (y_lim[1] - y_lim[0])/30
    # generate the mesh for plotting the plane
    xx, yy = np.meshgrid(np.arange(x_lim[0], x_lim[1], x_stp),
                         np.arange(y_lim[0], y_lim[1], y_stp))
    # get the normal vector of the plane
    for normal, dist in planes:
        # calculate the z-elevation of the planes
        zz = (-normal[0]*xx - normal[1]*yy + dist)/normal[2]
 
        ax.plot_surface(xx, yy, zz, color='b', alpha=0.05) # color=(1, 0, 0, 0.3)
    ax.set_xlim([-cubesize/10, cubesize/10])
    ax.set_ylim([-cubesize, cubesize])
    ax.set_zlim([-cubesize, cubesize])
    # plot vectors
    if vectors is not None:
        X = [origin[0], origin[0], origin[0]]
        Y = [origin[1], origin[1], origin[1]]
        Z = [origin[2], origin[2], origin[2]]
        U = [vectors[0][0], vectors[1][0], vectors[2][0]]
        V = [vectors[0][1], vectors[1][1], vectors[2][1]]
        W = [vectors[0][2], vectors[1][2], vectors[2][2]]
        ax.quiver(X, Y, Z, U, V, W)
    # show the plot
    plt.show()


def normal_basis(plane1, plane2, plane3):
    '''
    calculate a normal-basis from the intersections of 3 planes:
    b1, b2, b3, origin
    '''
    #print("------------")
    #print("basis vectors")
    b1 = np.cross(plane1[0], plane2[0])
    b1 = b1/np.linalg.norm(b1)
    b2 = np.cross(plane2[0], plane3[0])
    b2 = b2/np.linalg.norm(b2)
    b3 = np.cross(plane3[0], plane1[0])
    b3 = b3/np.linalg.norm(b3)

    matrix = np.vstack((plane1[0], plane2[0], plane3[0]))
    rhs = np.array((plane1[1], plane2[1], plane3[1]))
    origin = np.linalg.solve(matrix, rhs)

    #print("basis_1", b1)
    #print("basis_2", b2)
    #print("basis_3", b3)
    #print("origin", origin)

    return b1, b2, b3, origin


def new_coord(coord, b1, b2, b3, ori):
    '''
    get the new coordinates of a list of points for a given new coordiante-sys
    with basis vectors b1,b2,b3 and the origin ori (where the planes intersect)
    '''
    # the basis-change matrix
    mat = np.linalg.inv(np.vstack((b1, b2, b3)).T)
    new_coord = np.empty_like(coord)
    # calculate the new coords by multipling coord with the basis-change matrix
    for i in range(coord.shape[0]):
        new_coord[i] = np.squeeze(np.dot(mat,
                                         np.array(coord[i]-ori, ndmin=2).T))
    return new_coord

######################## 3 minimization routine

def mae_to_int(length, new_coord, component=0):
    '''
    calculate the mean absolut error to a grid with given length of the given
    "component"-th basis-vector
    '''
    arr = new_coord[:, component]/length
    return np.mean(np.abs(arr-np.rint(arr)))


def get_basis_lengths(coord, b1, b2, b3, ori):
    '''
    given a suitable basis in normal form, calculate fitting lengths of
    the basis-vectors, so that the coords are lying in an integer-grid
    '''
    # get the distances within the coords
    dist_coord = distances(coord)
    # use the histogram to find the minimum distance between the points
    min_dist = first_val(*np.histogram(dist_coord, bins=coord.shape[0]), bound =  bounds) /2
    # set a maximal range where the plane can move from the origin
    max_dist = np.max(dist_coord)/7.0
    #print("min_dist", min_dist)
    #print("max_dist", max_dist)
    new_coor = new_coord(coord, b1, b2, b3, ori)
    lengths = []
    # optimize over the length of each basis vector
    for i in range(3):
        #print("----------------------")
        #print("scaling basis vector", i)
        length = fminbound(mae_to_int, min_dist, max_dist,
                           args=(new_coor, i),
                           xtol=1e-5,
                           maxfun=50000,
                           disp=False)
        # check if a multiple of the actual length fits also
        # if bigger 
        error = mae_to_int(length, new_coor, i)
        #print("error", error)
        scale = 1
        for j in range(2, 5):
            error_i = mae_to_int(j*length, new_coor, i)
            if abs(error - error_i) > min_dist/2:
                #print("error_diff", abs(error - error_i))
                #print("scaling", j-1)
                scale = j-1
                break

        lengths.append(scale*length)

    return lengths

def get_combi_iter(iterat):
    ''' 
    given a suitable basis in normal form, calculate fitting lengths of
    the basis-vectors, so that the coords are lying in an integer-grid
    '''
    comb = combinations(range(iterat), 3)
    combi = [] 
    for i in list(comb):
        combi.append(i)
    return np.array(combi)

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """  
    
    if np.linalg.norm(v1) == 0 or np.linalg.norm(v2) == 0:
        return 0
    
    else:
        v1_u = unit_vector(v1)
        v2_u = unit_vector(v2)
        return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    

In [2]:
if __name__ == "__main__":
    # load the points
    coord = np.load("autocoi_thresh_50.npy")
    
    bounds = len(coord)/2

    ###########    USERINTERACTION ###############
    iterat = 4  # number of iterations (number of planes that are calculated)
                # estimate 4 planes in hessian normal form given by angles of norm-vector
                # (phi, psi) and distance to origin    
    
    planes = estimate_planes(coord, iterations=iterat);
    # plane 1 and 2 are very similar (take plane 2 in the following)
    # get the normal-basis from plane 0, 2 and 3
    
    

----- iteration: 0
----- iteration: 1
----- iteration: 2
----- iteration: 3


In [3]:
combi = get_combi_iter(iterat) # all combination of planes
sumbasvec = []
basangles = []

################# USERINTERACTION ################
combi_n = 3 # plane combination [1, 2, 3]


p1, p2, p3 = combi[combi_n]
b1, b2, b3, origin = normal_basis(planes[p1], planes[p2], planes[p3]);
scale = get_basis_lengths(coord, b1, b2, b3, origin);

# rescale the basis
basis1 = scale[0]*b1
basis2 = scale[1]*b2
basis3 = scale[2]*b3

# planes for the shortes sum of basis vectors
#p1, p2, p3 = combi[sumbasvec == np.min(sumbasvec)][0] # get plane combination with minimal sum of basis vecotrs


# plot the planes and basis-vectors

plot_planes(coord, [planes[p1], planes[p2], planes[p3]],
            vectors=[basis1, basis2, basis3],
            origin=origin)
# plot the coordinates in the integer grid
int_coord = np.empty_like(coord)


# get the new coordinates for the new basis
new_coor = new_coord(coord, basis1, basis2, basis3, origin)

# round new coordinates to integer values
new_coor = np.array(np.rint(new_coor), dtype=int)

for i in range(3):
    int_coord[:, i] = (new_coor[:, 0]*basis1[i] +
                       new_coor[:, 1]*basis2[i] +
                       new_coor[:, 2]*basis3[i] +
                       origin[i])

# basis angles
alpha1 = angle_between(basis1, basis2)
alpha2 = angle_between(basis1, basis3)
alpha3 = angle_between(basis2, basis3)




# save all results
np.savetxt("basis1.txt", basis1)
np.savetxt("basis2.txt", basis2)
np.savetxt("basis3.txt", basis3)
np.savetxt("origin.txt", origin)
np.savetxt("new_coord.txt", new_coor, fmt="%d")

plot_planes(int_coord, [planes[p1], planes[p2], planes[p3]],
            vectors=[basis1, basis2, basis3],
            origin=origin)

# Data Plot

In [43]:
#for ang1 in range(0, 3):
#    for ang2 in range(0, 3): 


plt.rc('text', usetex=True)
plt.rc('font', family='serif')

cubesize = 6
vectors = [basis1, basis2, basis3]
origin = origin

# create the figure for plotting
fig = plt.figure()
# create the first axis
ax = fig.add_subplot(111, projection='3d')

# plot the points
bg3d = ax.scatter(coord[:, 0], coord[:, 1], coord[:, 2], c='k' )
# get the axes-limits to cut the plane afterwards
x_lim = ax.get_xlim()
y_lim = ax.get_ylim()
z_lim = ax.get_zlim()

ax.set_xlabel("kz [1/nm]")
ax.set_ylabel("kx [1/nm]")
ax.set_zlabel("ky [1/nm]")

# set 30 steps for the plane in x and y direction
x_stp = (x_lim[1] - x_lim[0])/30
y_stp = (y_lim[1] - y_lim[0])/30
# generate the mesh for plotting the plane
xx, yy = np.meshgrid(np.arange(x_lim[0], x_lim[1], x_stp),
                     np.arange(y_lim[0], y_lim[1], y_stp))
# get the normal vector of the plane
for normal, dist in planes:
    # calculate the z-elevation of the planes
    zz = (-normal[0]*xx - normal[1]*yy + dist)/normal[2]

    ax.plot_surface(xx, yy, zz, color='b', alpha=0.075) # color=(1, 0, 0, 0.3)

ax.set_xlim([-cubesize/10, cubesize/10])
ax.set_ylim([-cubesize, cubesize])
ax.set_zlim([-cubesize, cubesize])
# plot vectors
if vectors is not None:
    X = [origin[0], origin[0], origin[0]]
    Y = [origin[1], origin[1], origin[1]]
    Z = [origin[2], origin[2], origin[2]]
    U = [vectors[0][0], vectors[1][0], vectors[2][0]]
    V = [vectors[0][1], vectors[1][1], vectors[2][1]]
    W = [vectors[0][2], vectors[1][2], vectors[2][2]]
    ax.quiver(X, Y, Z, U, V, W, color='r')
# show the plot
plt.show()

#figname = "data" + str(ang1) + "-" + str(ang2) + ".pdf"
#fig.savefig(figname, bbox_inches='tight')

# simulation plot

In [12]:


plt.rc('text', usetex=True)
plt.rc('font', family='serif')

cubesize = 6
vectors = [basis1, basis2, basis3]
origin = origin

# create the figure for plotting
fig = plt.figure()
# create the first axis
ax = fig.add_subplot(111, projection='3d')



# plot the points
bg3d = ax.scatter(int_coord[:, 0], int_coord[:, 1], int_coord[:, 2], c='k' )
# get the axes-limits to cut the plane afterwards
x_lim = ax.get_xlim()
y_lim = ax.get_ylim()
z_lim = ax.get_zlim()

ax.set_xlabel("kz [1/nm]")
ax.set_ylabel("kx [1/nm]")
ax.set_zlabel("ky [1/nm]")

# set 30 steps for the plane in x and y direction
x_stp = (x_lim[1] - x_lim[0])/30
y_stp = (y_lim[1] - y_lim[0])/30
# generate the mesh for plotting the plane
xx, yy = np.meshgrid(np.arange(x_lim[0], x_lim[1], x_stp),
                     np.arange(y_lim[0], y_lim[1], y_stp))
# get the normal vector of the plane
for normal, dist in planes:
    # calculate the z-elevation of the planes
    zz = (-normal[0]*xx - normal[1]*yy + dist)/normal[2]

    ax.plot_surface(xx, yy, zz, color='b', alpha=0.075) # color=(1, 0, 0, 0.3)

ax.set_xlim([-cubesize/10, cubesize/10])
ax.set_ylim([-cubesize, cubesize])
ax.set_zlim([-cubesize, cubesize])
# plot vectors
if vectors is not None:
    X = [origin[0], origin[0], origin[0]]
    Y = [origin[1], origin[1], origin[1]]
    Z = [origin[2], origin[2], origin[2]]
    U = [vectors[0][0], vectors[1][0], vectors[2][0]]
    V = [vectors[0][1], vectors[1][1], vectors[2][1]]
    W = [vectors[0][2], vectors[1][2], vectors[2][2]]
    ax.quiver(X, Y, Z, U, V, W, color='r')
    # show the plot
plt.show()

# data animation

In [8]:
from matplotlib import animation

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

cubesize = 6
vectors = [basis1, basis2, basis3]
origin = origin

# create the figure for plotting
fig = plt.figure()
# create the first axis
ax = fig.add_subplot(111, projection='3d')
# plot the points
bg3d = ax.scatter(coord[:, 0], coord[:, 1], coord[:, 2], c='k' )
# get the axes-limits to cut the plane afterwards
x_lim = ax.get_xlim()
y_lim = ax.get_ylim()
z_lim = ax.get_zlim()

ax.set_xlabel("kz [1/nm]")
ax.set_ylabel("kx [1/nm]")
ax.set_zlabel("ky [1/nm]")

# set 30 steps for the plane in x and y direction
x_stp = (x_lim[1] - x_lim[0])/30
y_stp = (y_lim[1] - y_lim[0])/30
# generate the mesh for plotting the plane
xx, yy = np.meshgrid(np.arange(x_lim[0], x_lim[1], x_stp),
                     np.arange(y_lim[0], y_lim[1], y_stp))
# get the normal vector of the plane
for normal, dist in planes:
    # calculate the z-elevation of the planes
    zz = (-normal[0]*xx - normal[1]*yy + dist)/normal[2]

    ax.plot_surface(xx, yy, zz, color='b', alpha=0.1) # color=(1, 0, 0, 0.3)

ax.set_xlim([-cubesize/10, cubesize/10])
ax.set_ylim([-cubesize, cubesize])
ax.set_zlim([-cubesize, cubesize])
# plot vectors
if vectors is not None:
    X = [origin[0], origin[0], origin[0]]
    Y = [origin[1], origin[1], origin[1]]
    Z = [origin[2], origin[2], origin[2]]
    U = [vectors[0][0], vectors[1][0], vectors[2][0]]
    V = [vectors[0][1], vectors[1][1], vectors[2][1]]
    W = [vectors[0][2], vectors[1][2], vectors[2][2]]
    ax.quiver(X, Y, Z, U, V, W)
# show the plot
plt.show()

def init():
    bg3d
    return fig,

def animate(i):
    ax.view_init(elev=i/2, azim=i)
    return fig,

# Set up formatting for the movie files
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Tillmann Wieland'), bitrate=50000)

# Animate
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=100,  blit=True, repeat_delay=3)# interval=1000,

anim.save('plane_data_animation.mp4', writer=writer)

# Simulated animation

In [9]:
from matplotlib import animation

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

cubesize = 6
vectors = [basis1, basis2, basis3]
origin = origin

# create the figure for plotting
fig = plt.figure()
# create the first axis
ax = fig.add_subplot(111, projection='3d')
# plot the points
bg3d = ax.scatter(int_coord[:, 0], int_coord[:, 1], int_coord[:, 2], c='k' )
# get the axes-limits to cut the plane afterwards
x_lim = ax.get_xlim()
y_lim = ax.get_ylim()
z_lim = ax.get_zlim()

ax.set_xlabel("kz [1/nm]")
ax.set_ylabel("kx [1/nm]")
ax.set_zlabel("ky [1/nm]")

# set 30 steps for the plane in x and y direction
x_stp = (x_lim[1] - x_lim[0])/30
y_stp = (y_lim[1] - y_lim[0])/30
# generate the mesh for plotting the plane
xx, yy = np.meshgrid(np.arange(x_lim[0], x_lim[1], x_stp),
                     np.arange(y_lim[0], y_lim[1], y_stp))
# get the normal vector of the plane
for normal, dist in planes:
    # calculate the z-elevation of the planes
    zz = (-normal[0]*xx - normal[1]*yy + dist)/normal[2]

    ax.plot_surface(xx, yy, zz, color='b', alpha=0.1) # color=(1, 0, 0, 0.3)

ax.set_xlim([-cubesize/10, cubesize/10])
ax.set_ylim([-cubesize, cubesize])
ax.set_zlim([-cubesize, cubesize])
# plot vectors
if vectors is not None:
    X = [origin[0], origin[0], origin[0]]
    Y = [origin[1], origin[1], origin[1]]
    Z = [origin[2], origin[2], origin[2]]
    U = [vectors[0][0], vectors[1][0], vectors[2][0]]
    V = [vectors[0][1], vectors[1][1], vectors[2][1]]
    W = [vectors[0][2], vectors[1][2], vectors[2][2]]
    ax.quiver(X, Y, Z, U, V, W)
# show the plot
plt.show()

def init():
    bg3d
    return fig,

def animate(i):
    ax.view_init(elev=i/2, azim=i)
    return fig,

# Set up formatting for the movie files
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Tillmann Wieland'), bitrate=50000)

# Animate
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=100,  blit=True, repeat_delay=3)# interval=1000,

anim.save('plane_simulated.mp4', writer=writer)

In [19]:
angb12 = angle_between(basis1,basis2)*180/np.pi
angb23 = angle_between(basis2,basis3)*180/np.pi
angb13 = angle_between(basis1,basis3)*180/np.pi
print("Angel between basis 12, 23, 13:")
print(angb12, angb23, angb13)


nb1 = np.linalg.norm(basis1)
nb2 = np.linalg.norm(basis2)
nb3 = np.linalg.norm(basis3)

print("nb1, nb2, nb3:")
print(nb1, nb2, nb3)

Angel between basis 12, 23, 13:
92.51315915408244 16.790909398742148 75.82213705292297
nb1, nb2, nb3:
1.2774142823217831 2.673145237644728 2.7123315542042348


In [21]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')


cubesize = 6
vectors = [basis1, basis2, basis3]
origin = int_coord[indices[0,0]]

# create the figure for plotting
fig = plt.figure()
# create the first axis
ax = fig.add_subplot(111, projection='3d')



# plot the points
bg3d = ax.scatter(int_coord[:, 0], int_coord[:, 1], int_coord[:, 2], c='k' )


ax.set_xlabel(r'kz in $\frac{1}{nm}$]')
ax.set_ylabel(r'kx in $\frac{1}{nm}$]')
ax.set_zlabel(r'ky in $\frac{1}{nm}$]')



limit1 = -1.5
limit2 = 1.5
ax.set_xlim([limit1, limit2])
ax.set_ylim([limit1, limit2])
ax.set_zlim([limit1, limit2])
# plot vectors
if vectors is not None:
    X = [origin[0], origin[0], origin[0]]
    Y = [origin[1], origin[1], origin[1]]
    Z = [origin[2], origin[2], origin[2]]
    U = [vectors[0][0], vectors[1][0], vectors[2][0]]
    V = [vectors[0][1], vectors[1][1], vectors[2][1]]
    W = [vectors[0][2], vectors[1][2], vectors[2][2]]
    ax.quiver(X, Y, Z, U, V, W, color='blue')
    # show the plot
plt.show()

# Smallest, linear independent basis vector ansatz

In [22]:
# import int_coord
int_coord= np.loadtxt("int_coord.txt")

kdt = KDTree(int_coord, leaf_size=2, metric='euclidean') # int_coord
distances, indices = kdt.query(int_coord, k=len(int_coord))

In [79]:
# make sure that the beginning point is the center of the point cloud
smdist = distances[0]
for i in range(len(distances)):
    while distances[i,j] <= smdist[j]:
            smdist = distances[i]
            smind = i
            i =+ 1


#find nearest neighbor of beginning point
for i in range(len(smdist)):
    if smdist[i] > 0:
        dist_min = smdist[i]
        break
        
indi_min = indices[smind][smdist == dist_min]
new_basis1 = int_coord[indi_min][0] - int_coord[indices[smind,0]] # first basis vector

#find second nearest neighbor of first lattice point
for i in range(len(smdist)):
    if smdist[i] > dist_min:
        dist_min2 = smdist[i]
        
        indi_min2 = indices[smind][smdist == dist_min2]
        new_basis2 = int_coord[indi_min2][0] - int_coord[indices[smind,0]] # second basis vector
        
        #check if linear independent      
        A = np.row_stack([new_basis1, new_basis2])
        U, s, V = np.linalg.svd(A)
        
        if np.sum(s > 0.0000001) > 1 :      # rank of matrix must be bigger than 1 for the vectors to be
                                            # linear independent
            break

#find third nearest neighbor of first lattice point
for i in range(len(distances[0])):
    if smdist[i] > dist_min2:
        dist_min3 = smdist[i]
        
        indi_min3 = indices[smind][smdist == dist_min3]
        new_basis3 = int_coord[indi_min3][0] - int_coord[indices[smind,0]] # third basis vector
        #check if linear independent      
        A = np.row_stack([new_basis1, new_basis2, new_basis3])
        U, s, V = np.linalg.svd(A)
        
        if np.sum(s > 0.0001) > 2:          # rank of matrix must be bigger than 2 for the vectors to be
                                            # linear independent
            break

In [82]:
angnb12 = angle_between(new_basis1,new_basis2)*180/np.pi
angnb23 = angle_between(new_basis2,new_basis3)*180/np.pi
angnb13 = angle_between(new_basis1,new_basis3)*180/np.pi
print("Angel between basis 12, 23, 13:")
print(angnb12, angnb23, angnb13)


nb1 = np.linalg.norm(new_basis1)
nb2 = np.linalg.norm(new_basis2)
nb3 = np.linalg.norm(new_basis3)

print("nb1, nb2, nb3:")
print(nb1, nb2, nb3)

Angel between basis 12, 23, 13:
86.99949098196304 77.2033649192605 99.95888772001764
nb1, nb2, nb3:
0.34267448893584246 0.3531766025541934 2.4959732472471834


In [112]:
A = np.row_stack([np.array([1,1,0]), np.array([0,0,1])])
U, s, V = np.linalg.svd(A)

In [115]:
A

array([[1, 1, 0],
       [0, 0, 1]])

In [114]:
U

array([[1., 0.],
       [0., 1.]])

In [113]:
s

array([1.41421356, 1.        ])

In [116]:
V

array([[ 0.70710678,  0.70710678,  0.        ],
       [ 0.        ,  0.        ,  1.        ],
       [ 0.70710678, -0.70710678,  0.        ]])

# Plot of new basis

In [89]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')


cubesize = 6
vectors = [new_basis1, new_basis2, new_basis3]
origin = int_coord[smind]

# create the figure for plotting
fig = plt.figure()
# create the first axis
ax = fig.add_subplot(111, projection='3d')



# plot the points
bg3d = ax.scatter(int_coord[:, 0], int_coord[:, 1], int_coord[:, 2], c='k' )


ax.set_xlabel(r'kz in $\frac{1}{nm}$]')
ax.set_ylabel(r'kx in $\frac{1}{nm}$]')
ax.set_zlabel(r'ky in $\frac{1}{nm}$]')



limit1 = -1.5
limit2 = 1
ax.set_xlim([limit1, limit2])
ax.set_ylim([limit1, limit2])
ax.set_zlim([limit1, limit2])
# plot vectors
if vectors is not None:
    X = [origin[0], origin[0], origin[0]]
    Y = [origin[1], origin[1], origin[1]]
    Z = [origin[2], origin[2], origin[2]]
    U = [vectors[0][0], vectors[1][0], vectors[2][0]]
    V = [vectors[0][1], vectors[1][1], vectors[2][1]]
    W = [vectors[0][2], vectors[1][2], vectors[2][2]]
    ax.quiver(X, Y, Z, U, V, W, color='blue')
    # show the plot
plt.show()

# Histogram neighbor distance of Data COI

In [ ]:
#Version bigger bins

from sklearn.neighbors import KDTree
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

b1 = 0.36
b3 = 2.54


# Use Latex
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

#find the nearest neighbor and its distance

plotcoords = coord # int_coord

kdt = KDTree(plotcoords, leaf_size=2, metric='euclidean') # int_coord
d_distances, indices = kdt.query(plotcoords, k=len(plotcoords))

# delete the nearest neighbor distance to itself
d_distances = np.delete(d_distances.T, 0, 0).T

fig, ax = plt.subplots(figsize=(20,10))
plt.hist(d_distances.ravel(),bins=2000, facecolor='k')
plt.xlabel(r'Reciprocal Neighbor Distances [$\frac{1}{nm}$]', fontsize = 25)
plt.ylabel('Occurrence', fontsize = 30)
xtick = np.round(np.arange(0.0, 4, 0.2), decimals=1)
labels = xtick
# You can specify a rotation for the tick labels in degrees or with keywords.
plt.xticks(xtick, labels, fontsize=15 )
plt.yticks(fontsize=20 )
plt.axis([0, 2.8, 0, 1300])
plt.grid(True)
plt.axvline(x=b1, color = "r", ls= "dashed", lw = "2")
plt.axvline(x=np.sqrt(2)*b1, color = "r", ls= "dashed", lw = "2")
plt.axvline(x=np.sqrt(5)*b1, color = "r", ls= "dashed", lw = "2")

fs = 28
plt.text(b1 -0.05, 1100, r"$b \textsubscript{1,2} $" , fontsize=fs,  bbox=dict(boxstyle="Square,pad=0.21", ec="r", facecolor='white', alpha=1))
plt.text(np.sqrt(2)*b1 -0.1, 800, r"$\sqrt{2} \hspace{0.1cm}  b\textsubscript{1,2} $" , fontsize=fs,  bbox=dict(boxstyle="Square,pad=0.21", ec="r", facecolor='white', alpha=1))
plt.text(np.sqrt(5)*b1 -0.09, 1150, r"$\sqrt{5} \hspace{0.1cm}   b\textsubscript{1,2} $" , fontsize=fs,  bbox=dict(boxstyle="Square,pad=0.21", ec="r", facecolor='white', alpha=1))
plt.text(b3 -0.1, 1100, r"$b \textsubscript{3} $" , fontsize=fs,  bbox=dict(boxstyle="Square,pad=0.25", ec="r", facecolor='white', alpha=1))




plt.axvline(x=b3, color = "r", ls= "dashed", lw = "2")
minorLocator = MultipleLocator(5)

ax.xaxis.set_minor_locator(minorLocator)
plt.show()

# Histogram neighbor distance of Simulation COI

In [ ]:
from sklearn.neighbors import KDTree
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

b1 = 0.36
b3 = 2.54


# import int_coord
int_coord= np.loadtxt("int_coord.txt")


# Use Latex
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

#find the nearest neighbor and its distance

plotcoords = int_coord

kdt = KDTree(plotcoords, leaf_size=2, metric='euclidean') # int_coord
sim_distances, indices = kdt.query(plotcoords, k=len(plotcoords))

# delete the nearest neighbor distance to itself
sim_distances = np.delete(sim_distances.T, 0, 0).T

fig, ax = plt.subplots(figsize=(20,10))
plt.hist(sim_distances.ravel(),bins=2000, facecolor='k')
plt.xlabel(r'Reciprocal Neighbor Distances [$\frac{1}{nm}$]', fontsize = 25)
plt.ylabel('Occurrence', fontsize = 30)
xtick = np.round(np.arange(0.0, 4, 0.2), decimals=1)
labels = xtick
# You can specify a rotation for the tick labels in degrees or with keywords.
plt.xticks(xtick, labels, fontsize=15 )
plt.yticks(fontsize=20 )
plt.axis([0, 2.8, 0, 3000])
plt.grid(True)
plt.axvline(x=b1, color = "r", ls= "dashed", lw = "2")
plt.axvline(x=np.sqrt(2)*b1, color = "r", ls= "dashed", lw = "2")
plt.axvline(x=np.sqrt(5)*b1, color = "r", ls= "dashed", lw = "2")

fs = 30
plt.text(b1 -0.1, 2700, r"$b \textsubscript{1,2} $" , fontsize=fs,  bbox=dict(boxstyle="Square,pad=0.21", ec="r", facecolor='white', alpha=1))
plt.text(np.sqrt(2)*b1 -0.1, 2200, r"$\sqrt{2} \hspace{0.1cm}  b\textsubscript{1,2} $" , fontsize=fs,  bbox=dict(boxstyle="Square,pad=0.21", ec="r", facecolor='white', alpha=1))
plt.text(np.sqrt(5)*b1 -0.09, 1700, r"$\sqrt{5} \hspace{0.1cm}  b\textsubscript{1,2} $" , fontsize=fs,  bbox=dict(boxstyle="Square,pad=0.21", ec="r", facecolor='white', alpha=1))
plt.text(b3 -0.04, 2600, r"$b \textsubscript{3} $" , fontsize=fs,  bbox=dict(boxstyle="Square,pad=0.25", ec="r", facecolor='white', alpha=1))




plt.axvline(x=b3, color = "r", ls= "dashed", lw = "2")
minorLocator = MultipleLocator(5)

ax.xaxis.set_minor_locator(minorLocator)
plt.show()


# Angles of nearest neighbor

In [28]:
from tqdm import tqdm
from sklearn.neighbors import KDTree
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

# import int_coord
int_coord= np.loadtxt("int_coord.txt")

#save int_coord
#np.savetxt("int_coord.txt", int_coord)


kdt = KDTree(int_coord, leaf_size=2, metric='euclidean') # int_coord
distances, indices = kdt.query(int_coord, k=len(int_coord))

# init
sort_coor = np.zeros([indices.shape[0],indices.shape[1],3])
diff_vec = np.zeros([indices.shape[0],indices.shape[1],3])

for z in range(distances.shape[0]) :
    for s in range(distances.shape[1]):
        #sorted coords
        sort_coor[z,s] = int_coord[indices[z,s]]
        diff_vec[z,s] = sort_coor[z,0] - sort_coor[z,s]
        




ang = list()

for j in tqdm(range(diff_vec.shape[0])):

    diff_vec_nearest = diff_vec[j,2:22]
    sort_coor_nearest = sort_coor[j,2:22]
    for i in range(diff_vec_nearest.shape[0]):
            angt = angle_between(diff_vec_nearest[0],diff_vec_nearest[i])
            if not angt == 0:
                ang.append(angt)
                
angdeg = np.array(ang)*180/np.pi





100%|██████████| 851/851 [00:01<00:00, 736.48it/s]


In [6]:


fig, ax = plt.subplots(figsize=(20,10))
plt.hist(ang ,bins=200, facecolor='k')
plt.xlabel(r'Nearest Neighbor Angles', fontsize = 25)
plt.ylabel('Occurrence', fontsize = 25)
xtick = np.round(np.arange(0.0, np.pi , np.pi/4 ), decimals=2)
labels = xtick
# You can specify a rotation for the tick labels in degrees or with keywords.
plt.xticks(xtick, labels, fontsize=18 )
plt.yticks(fontsize=18 )
plt.axis([0, np.pi, 0, 1300])
#plt.grid(True)
plt.axvline(x=np.pi/4, color = "r", ls= "dashed", lw = "2")
plt.axvline(x=np.pi/2, color = "r", ls= "dashed", lw = "2")
plt.axvline(x=np.pi*3/4, color = "r", ls= "dashed", lw = "2")

fs = 35
plt.text(np.pi/4 - 0.03, 600, r"$ \frac{\pi}{4}$" , fontsize=fs,  bbox=dict(boxstyle="Square,pad=0.3", ec="r", facecolor='white', alpha=1))
plt.text(np.pi/2- 0.03, 600, r"$ \frac{\pi}{2} $" , fontsize=fs,  bbox=dict(boxstyle="Square,pad=0.3", ec="r", facecolor='white', alpha=1))
plt.text(np.pi*3/4 - 0.03, 600, r"$ \frac{3 \hspace{0.1cm} \pi}{4} $" , fontsize=fs,  bbox=dict(boxstyle="Square,pad=0.3", ec="r", facecolor='white', alpha=1))
#plt.text(b3 -0.1, 1100, r"$b \textsubscript{3} $" , fontsize=fs,  bbox=dict(boxstyle="Square,pad=0.25", ec="r", facecolor='white', alpha=1))




Text(2.32619,600,'$ \\frac{3 \\hspace{0.1cm} \\pi}{4} $')

# Volume of basis-cell with triple product (slow)

In [17]:
basis1 = np.loadtxt("basis1.txt")
basis2 = np.loadtxt("basis2.txt")
basis3 = np.loadtxt("basis3.txt")

new_basis = list()

linrange = 10

for i in range(-linrange,linrange):
    for j in range(-linrange,linrange):
        for k in range(-linrange,linrange):
            
            new_basis_temp = i*basis1 + j*basis2 + k*basis3
            new_basis.append(new_basis_temp)
            
def tripleproduct(a,b,c):
    t=0
    t=\
    a[0]*(b[1]*c[2]-b[2]*c[1])+\
    a[1]*(b[2]*c[0]-b[0]*c[2])+\
    a[2]*(b[0]*c[1]-b[1]*c[0])  # 14 operations / det
    return t

lin_combi_vol = list()
lin_combi_indi = list()
lin_combi_ang = list()


for i in tqdm_notebook(range(len(new_basis))):
    for j in range(len(new_basis)):
        for k in range(len(new_basis)):
            
            if i!=j or i!=k or j!=k:
                # test if angle between vectors greater than angborder
                angborder = 0.3

                new_basis1 = new_basis[i]
                new_basis2 = new_basis[j]
                new_basis3 = new_basis[k]

                #ang1 = np.abs(angle_between(new_basis1,new_basis2))
                #ang2 = np.abs(angle_between(new_basis1,new_basis3))
                #ang3 = np.abs(angle_between(new_basis2,new_basis3))



                #if ang1 > angborder and ang2 > angborder and ang3 > angborder:
                    
                   
                lin_combi_vol_temp = tripleproduct(new_basis1,new_basis2,new_basis3)
                lin_combi_indi.append([i,j,k])
                #lin_combi_ang.append([ang1, ang2, ang3])
                lin_combi_vol.append(lin_combi_vol_temp)
                
# find indices of smallest non-zero basis-cell-volume
lin_combi_vol_abs = np.abs(lin_combi_vol)


504

In [ ]:

for i in range(len(lin_combi_vol_abs)):
    if lin_combi_vol_abs[i] 


In [18]:
np.array(lin_combi_vol).shape

(504,)

In [10]:

                
# basis_cell volume bigger than volborder
lin_combi_vol_abs_nonzero = lin_combi_vol_abs[lin_combi_vol_abs > volborder]

lin_indi_coord_diff = list()

lin_combi_vol_sort = np.argsort(lin_combi_vol_abs_nonzero)

# test if coordinate difference is bigger than dnbborder
dnbborder = 0.3

In [11]:


for i in tqdm_notebook(range(len(lin_combi_vol_abs_nonzero))):

    lin_combi_min = lin_combi_vol_abs_nonzero[lin_combi_vol_sort[i]]
    lin_combi_min_indi = np.array(lin_combi_indi)[lin_combi_vol_abs == lin_combi_min]
    

    # test if coordinates are similar to each other
    new_basis1 = new_basis[lin_combi_min_indi[0,0]]
    new_basis2 = new_basis[lin_combi_min_indi[0,1]]
    new_basis3 = new_basis[lin_combi_min_indi[0,2]]

    # x-coordinates
    dnbx1 = np.abs(new_basis1[0]-new_basis2[0])
    dnbx2 = np.abs(new_basis1[0]-new_basis3[0])

    # y-coordinates
    dnby1 = np.abs(new_basis1[1]-new_basis2[1])
    dnby2 = np.abs(new_basis1[1]-new_basis3[1])

    # z-coordinates
    dnbz1 = np.abs(new_basis1[2]-new_basis2[2])
    dnbz2 = np.abs(new_basis1[2]-new_basis3[2])
 
    if dnbx1 > dnbborder or dnbx2 > dnbborder:
        if dnby1 > dnbborder or dnby2 > dnbborder:
            if dnbz1 > dnbborder or dnbz2 > dnbborder:
                lin_indi_coord_diff.append(list(lin_combi_min_indi[0]))
        

KeyboardInterrupt: 

In [268]:
lin_indi_coord_diff
    
    
    

[]

In [236]:
lin_indi_coord_diff

[]

In [224]:
lin_indi_coord_diff.append(list(lin_combi_min_indi[0]))

In [225]:
lin_indi_coord_diff

[[0, 6, 62], [0, 6, 62]]

In [204]:
dnbx1

-0.007394737967465356

In [200]:
new_basis1[1]-new_basis2[1]

-4.7465374299908785

In [201]:
new_basis1[1]-new_basis3[1]

-5.975678767956548

In [89]:
lin_combi_min_angles = np.array(lin_combi_ang)[lin_combi_vol_abs == lin_combi_vol_min]
lin_combi_min_indi = np.array(lin_combi_indi)[lin_combi_vol_abs == lin_combi_vol_min]

In [90]:
lin_combi_min_indi

array([[56,  1,  7],
       [56,  7,  1]])

In [177]:
new_basis1 = new_basis[56]
new_basis2 = new_basis[1]
new_basis3 = new_basis[7]

In [182]:
new_basis1

array([ 0.05821497, -3.68009936, -3.75735177])

In [179]:
new_basis2

array([-0.10728108, -3.52105842, -7.74721809])

In [180]:
new_basis3

array([-0.09988635,  1.22547901, -1.28128081])

In [94]:
def cart2sph(x, y, z):
    hxy = np.hypot(x, y)
    r = np.hypot(hxy, z)
    el = np.arctan2(z, hxy)
    az = np.arctan2(y, x)
    return az, el, r

In [99]:
cart2sph(new_basis1[0],new_basis1[1],new_basis1[2])

(-1.55497878662772, -0.7957221988579319, 5.25967799952975)

In [101]:
cart2sph(new_basis2[0],new_basis2[1],new_basis2[2])

(-1.6012553230147972, -1.1440373023036117, 8.510508193141156)

In [100]:
cart2sph(new_basis3[0],new_basis3[1],new_basis3[2])

(1.652124543914163, -0.806001139443553, 1.7757974544458028)

In [109]:
cart2sph(1,1,1)

(0.7853981633974483, 0.6154797086703873, 1.7320508075688774)

In [110]:
cart2sph(1,1.5,1)

(0.982793723247329, 0.5064446434135005, 2.0615528128088303)

In [105]:
cart2sph(3,2,0)

(0.5880026035475675, 0.0, 3.605551275463989)

In [120]:
lin_combi_vol_abs[np.argsort(lin_combi_vol_abs) == 0]

array([0.86651312])

In [92]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.gca(projection='3d')

ax.quiver(0, 0, 0, new_basis1, new_basis2, new_basis3, length=0.1, normalize=True)

plt.show()

In [133]:
np.linalg.norm(new_basis3)

14.32916602016006

In [154]:
lin_cross = np.cross(np.array(new_basis1),np.array(new_basis2))
lin_combi_vol_temp = np.dot(np.array(new_basis3),lin_cross)

In [155]:
lin_combi_vol_temp

0.2888377071501367

In [156]:
test1 = [0.36,0,0]
test2 = [0,0.36,0]
test3 = [0,0,2.54]


In [157]:
lin_cross = np.cross(np.array(test1),np.array(test2))
lin_combi_vol_temp = np.dot(np.array(test3),lin_cross)

In [158]:
lin_combi_vol_temp

0.329184

In [66]:
'''
==============
3D scatterplot
==============

Demonstration of a basic scatterplot in 3D.
'''

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


# For each set of style and range settings, plot n random points in the box
# defined by x in [23, 32], y in [0, 100], z in [zlow, zhigh].
ax.scatter(sort_coor_nearest.T[0],sort_coor_nearest.T[1],sort_coor_nearest.T[2])

ax.set_xlim([-0.2,0.4])
ax.set_ylim([0.1,-0.5])

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()


In [65]:
sort_coor_nearest

array([[-0.24876506,  0.09913626,  6.88072501],
       [-0.15921701, -0.22260796,  7.25926345],
       [ 0.00954081,  0.0376384 ,  7.11359794],
       [-0.32797483, -0.48285431,  7.40492895],
       [-0.32797483, -0.48285431,  7.40492895],
       [-0.08000725,  0.35938262,  6.73505951]])

In [ ]:
from tqdm import tqdm
from sklearn.neighbors import KDTree
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

# import int_coord
int_coord= np.loadtxt("int_coord.txt")

#save int_coord
#np.savetxt("int_coord.txt", int_coord)


kdt = KDTree(int_coord, leaf_size=2, metric='euclidean') # int_coord
distances, indices = kdt.query(int_coord, k=len(int_coord))

# init
sort_coor = np.zeros([indices.shape[0],indices.shape[1],3])
diff_vec = np.zeros([indices.shape[0],indices.shape[1],3])

for z in range(distances.shape[0]) :
    for s in range(distances.shape[1]):
        #sorted coords
        sort_coor[z,s] = int_coord[indices[z,s]]
        diff_vec[z,s] = sort_coor[z,0] - sort_coor[z,s]
        
ang = list()
for i in tqdm(range(len(diff_vec[0]))):
    for j in range(len(diff_vec[0])):
        angt = angle_between(diff_vec[0,i],diff_vec[0,j])
        if not angt == 0:
            ang.append(angt)

#save ang
np.savetxt("ang.txt", ang)

fig, ax = plt.subplots(figsize=(20,10))
plt.hist(ang,bins=200, facecolor='k')
plt.show()

# angle between all nearest neighbors der reihe nach

In [119]:
# import int_coord
int_coord= np.loadtxt("int_coord.txt")

from tqdm import tqdm
from sklearn.neighbors import KDTree
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

kdt = KDTree(int_coord, leaf_size=2, metric='euclidean') # int_coord
a_distances, a_indices = kdt.query(int_coord, k=len(int_coord))

# distances and indices of nearest neighbors of one point p 

############################### test #################
p = 40

p_distances = a_distances[p,0:10]
p_indices = a_indices[p,0:10]


# initialise 
sort_coor = np.zeros([p_indices.shape[0],3])


for i in range(p_indices.shape[0]) :
    sort_coor[i] = int_coord[p_indices[i]]
    
diff_vec = sort_coor

In [115]:
for i in range(sort_coor.shape[0]) :
    diff_vec[i] = sort_coor[0] - sort_coor[i]

In [105]:
sort_coor[0] - sort_coor[1]

array([-0.16875782, -0.26024636,  0.1456655 ])

In [85]:
ang = list()
for i in tqdm(range(len(diff_vec))):
    for j in range(len(diff_vec)):
        angt = angle_between(diff_vec[i],diff_vec[j])
        if not angt == 0:
            ang.append(angt)


#save ang
np.savetxt("ang.txt", ang)

fig, ax = plt.subplots(figsize=(20,10))
plt.hist(ang,bins=200, facecolor='k')
plt.show()




100%|██████████| 10/10 [00:00<00:00, 1665.40it/s]


In [47]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


# For each set of style and range settings, plot n random points in the box
# defined by x in [23, 32], y in [0, 100], z in [zlow, zhigh].
ax.scatter(sort_coor.T[0],sort_coor.T[1],sort_coor.T[2])

ax.axes

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()


In [77]:
p_distances = distances[5,0:20]
plt.plot(list(range(0, 20)), p_distances, 'ro')
plt.axis([0, 20, 0, p_distances.max()])
plt.show()

In [72]:
for i in range(10):
    p_distances = distances[i,0:20]
    plt.plot(list(range(0, 20)), p_distances, 'ro')
    plt.axis([0, 20, 0, p_distances.max()])
    plt.show()

# Angle between smallest Basis vectors of simulation

In [123]:
from tqdm import tqdm
from sklearn.neighbors import KDTree
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

# import int_coord
int_coord= np.loadtxt("int_coord.txt")

#save int_coord
#np.savetxt("int_coord.txt", int_coord)


kdt = KDTree(int_coord, leaf_size=2, metric='euclidean') # int_coord
distances, indices = kdt.query(int_coord, k=len(int_coord))

# init
sort_coor = np.zeros([indices.shape[0],indices.shape[1],3])
diff_vec = np.zeros([indices.shape[0],indices.shape[1],3])

for z in range(distances.shape[0]) :
    for s in range(distances.shape[1]):
        #sorted coords
        sort_coor[z,s] = int_coord[indices[z,s]]
        diff_vec[z,s] = sort_coor[z,0] - sort_coor[z,s]
        
ang = list()
for i in tqdm(range(len(diff_vec[0]))):
    for j in range(len(diff_vec[0])):
        angt = angle_between(diff_vec[0,i],diff_vec[0,j])
        if not angt == 0:
            ang.append(angt)

#save ang
np.savetxt("ang.txt", ang)

fig, ax = plt.subplots(figsize=(20,10))
plt.hist(ang,bins=200, facecolor='k')
plt.show()

100%|██████████| 851/851 [00:38<00:00, 22.23it/s]


In [22]:
plt.plot(distances[0], 'ro')
plt.axis([0, 20, 0, 2])
plt.show()

In [5]:
diff_vec.shape

(851, 851, 3)

# Angle between smallest vectors

In [3]:
from tqdm import tqdm
from sklearn.neighbors import KDTree
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

kdt = KDTree(int_coord, leaf_size=2, metric='euclidean') # int_coord
distances, indices = kdt.query(int_coord, k=len(int_coord))


######################### Threshhold ######################
lenthresh = 0.4
distmask = distances < lenthresh # 



# init
sort_coor = np.zeros([indices.shape[0],indices.shape[1],3])
diff_vec = np.zeros([indices.shape[0],indices.shape[1],3])

for z in range(distances.shape[0]) :
    for s in range(distances.shape[1]):
        #sorted coords
        sort_coor[z,s] = int_coord[indices[z,s]]
        diff_vec[z,s] = sort_coor[z,0] - sort_coor[z,s]

smvec = diff_vec[distmask]
smmask = np.linalg.norm(smvec, axis=1) == 0
smvec = smvec[~smmask]

ang = list()
for i in tqdm(range(len(smvec))):
    for j in range(len(smvec)):
        angt = angle_between(smvec[i],smvec[j])
        if not angt ==0:
            ang.append(angt)

            

        
            
            
            
        
            

100%|██████████| 2394/2394 [04:56<00:00,  8.07it/s]


In [12]:
#save ang
np.savetxt("ang.txt", ang)

In [5]:
fig, ax = plt.subplots(figsize=(20,10))
plt.hist(ang,bins=200, facecolor='k')
plt.show()

# Linear Combination of Basis Vectors

In [79]:
looplen = 3
blinear = []
for i in range(-looplen,looplen+1):
    if i not in [0]:
        for j in range(-looplen,looplen+1):
            if j not in [0]:
                for k in range(-looplen,looplen+1):
                    if k not in [0]:
                        blineari = i*basis1 + j*basis2 + k*basis3
                        blinear.append(blineari)
blinear = np.array(blinear)


In [8]:
int_coord

array([[ 0.02640261, -0.02752178, -0.25472332],
       [-0.41752288, -0.16111009,  7.02639051],
       [-0.41752288, -0.16111009,  7.02639051],
       ...,
       [ 0.41537489,  4.20545319, -7.91401719],
       [ 0.47032811,  0.10606653, -7.53583715],
       [ 0.47032811,  0.10606653, -7.53583715]])

In [82]:
b1len = np.linalg.norm(basis1)
b2len = np.linalg.norm(basis2)
b3len = np.linalg.norm(basis3)
blinearlen = np.linalg.norm(blinear,axis=1)
#blinmask = ((blinearlen == blinearlen.min()) & (blinearlen != 0))
#blinear[blinmask]

In [31]:
blinearlen == np.sort(blinearlen)[2]
b1shortest = [-0.1680066 , -0.21860188,  0.15164803]
blinmask = (blinearlen == np.sort(blinearlen)[2]) 
b2shortest = blinear[blinmask][0]
angle_between(b1shortest,b2shortest)


1.31745258366256

# get histogram nearest neighbors of KNBO from the literature

In [10]:
import itertools

permi = np.array(list(itertools.product(range(9),repeat = 3)))
lit_b1 = np.array([1/2.75,0,0])
lit_b2 = np.array([0,1/2.75,0])
lit_b3 = np.array([0,0,1/0.394])
lit_points = permi *lit_b1 + permi*lit_b2 + permi * lit_b3 


In [11]:
#find the nearest neighbor and its distance
kdt_lit = KDTree(lit_points, leaf_size=2, metric='euclidean')
distances_lit, indices_lit = kdt_lit.query(lit_points, k=100)

# delete the nearest neighbor distance to itself
distances_lit = distances_lit.T[1:].T

fig = plt.figure(figsize=(10,10))
plt.hist(distances_lit.ravel(),bins = 500, color = "r")
plt.hist(distances.ravel(), bins = 500, color = "b")
plt.show()

In [168]:
1/0.394

2.5380710659898478

In [24]:
fig = plt.figure()
# create the first axis
ax = fig.add_subplot(111, projection='3d')
# plot the points
bg3d = ax.scatter(lit_points.T[0], lit_points.T[1], lit_points.T[2] , c='k' )
# get the axes-limits to cut the plane afterwards
ax.set_xlabel("kx [1/nm]")
ax.set_ylabel("ky [1/nm]")
ax.set_zlabel("kz [1/nm]")

Text(0.5,0,'kz [1/nm]')

In [35]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

cubesize = 6
vectors = [b1shortest,b2shortest, b2shortest]
origin = origin

# create the figure for plotting
fig = plt.figure()
# create the first axis
ax = fig.add_subplot(111, projection='3d')
# plot the points
bg3d = ax.scatter(int_coord[:, 0], int_coord[:, 1], int_coord[:, 2], c='k' )
# get the axes-limits to cut the plane afterwards
x_lim = ax.get_xlim()
y_lim = ax.get_ylim()
z_lim = ax.get_zlim()

ax.set_xlabel("kz [1/nm]")
ax.set_ylabel("kx [1/nm]")
ax.set_zlabel("ky [1/nm]")

# set 30 steps for the plane in x and y direction
x_stp = (x_lim[1] - x_lim[0])/30
y_stp = (y_lim[1] - y_lim[0])/30
# generate the mesh for plotting the plane
xx, yy = np.meshgrid(np.arange(x_lim[0], x_lim[1], x_stp),
                     np.arange(y_lim[0], y_lim[1], y_stp))
# get the normal vector of the plane
for normal, dist in planes:
    # calculate the z-elevation of the planes
    zz = (-normal[0]*xx - normal[1]*yy + dist)/normal[2]

    ax.plot_surface(xx, yy, zz, color='b', alpha=0.1) # color=(1, 0, 0, 0.3)

ax.set_xlim([-cubesize/10, cubesize/10])
ax.set_ylim([-cubesize, cubesize])
ax.set_zlim([-cubesize, cubesize])
# plot vectors
if vectors is not None:
    X = [origin[0], origin[0], origin[0]]
    Y = [origin[1], origin[1], origin[1]]
    Z = [origin[2], origin[2], origin[2]]
    U = [vectors[0][0], vectors[1][0], vectors[2][0]]
    V = [vectors[0][1], vectors[1][1], vectors[2][1]]
    W = [vectors[0][2], vectors[1][2], vectors[2][2]]
    ax.quiver(X, Y, Z, U, V, W)
# show the plot
plt.show()

In [83]:
b1, b2, b3, origin = normal_basis(planes[p1], planes[p2], planes[p3]);
scale = get_basis_lengths(coord, b1, b2, b3, origin);

# rescale the basis
basis1 = scale[0]*b1
basis2 = scale[1]*b2
basis3 = scale[2]*b3

# planes for the shortes sum of basis vectors
#p1, p2, p3 = combi[sumbasvec == np.min(sumbasvec)][0] # get plane combination with minimal sum of basis vecotrs


# plot the planes and basis-vectors

plot_planes(coord, [planes[p1], planes[p2], planes[p3]],
            vectors=[basis1, basis2, basis3],
            origin=origin)
# plot the coordinates in the integer grid
int_coord = np.empty_like(coord)


# get the new coordinates for the new basis
new_coor = new_coord(coord, basis1, basis2, basis3, origin)

# round new coordinates to integer values
new_coor = np.array(np.rint(new_coor), dtype=int)

for i in range(3):
    int_coord[:, i] = (new_coor[:, 0]*basis1[i] +
                       new_coor[:, 1]*basis2[i] +
                       new_coor[:, 2]*basis3[i] +
                       origin[i])

# basis angles
alpha1 = angle_between(basis1, basis2)
alpha2 = angle_between(basis1, basis3)
alpha3 = angle_between(basis2, basis3)




# save all results
np.savetxt("basis1.txt", basis1)
np.savetxt("basis2.txt", basis2)
np.savetxt("basis3.txt", basis3)
np.savetxt("origin.txt", origin)
np.savetxt("new_coord.txt", new_coor, fmt="%d")

plot_planes(int_coord, [planes[p1], planes[p2], planes[p3]],
            vectors=[basis1, basis2, basis3],
            origin=origin)


TypeError: 'numpy.ndarray' object is not callable